## Background

---

This is the first of a series of two notebooks, where we forecast sales for a retailers with 45 stores, each contains several departments, in multiple regions. We also want to understand the extent to which external factors, including holidays and promotions, affect sales in different departments, stores, and regions. In this notebook, we are going to focus on the exploration and manipulation of our data, whereas the next one would be centered around modeling itself. Understanding the data is important for analytics, and we recommend you to read both notebooks in order to gain a better grasp of analytics for big and messy data. The three datasets that we are using are already included (see the "Data" folder) in the repository where this Jupyter notebook is located, but you can also find them on [this Kaggle page](https://www.kaggle.com/manjeetsingh/retaildataset?select=sales+data-set.csv). 

One of the most commonly used time series forecasting model are ARIMA models, which is what we are going to use in this notebook. However, there are also other models that you could use, such as ETS Models. At the same time, since we want to understand the relationships between sales and several other variables, including mark downs, holidays, and potentially weather (temperature), it would be benefitial for us to build a Simple or Multiple Linear Regression Model first, with sales as the response variable. Then we can build time series models to forecast the explanatory variables, such as temperature, and plug in the time-series predicted explantory variables into the SLR/MLR models to generate further forecasts for the response variable,sales. Models generated in this manner might be less accurate as we are going through multiple steps, each adding more risks for errors; however, time series models combined with explanatory models are more interpretable and provide you with more managerial insights that could help you make recommendations and decisions. Let's go ahead and import the libraries we need and load our data:

In [29]:
# basic tools for data analysis and visualization in Python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ipywidgets are used to make interactive contents in Jupyter notebooks
from ipywidgets import interact

In [30]:
df_sales = pd.read_csv('Data/sales.csv')
df_features = pd.read_csv('Data/features.csv')
df_stores = pd.read_csv('Data/stores.csv')

# converting the date column (initally stored as strings) to dates
df_sales['Date'] = pd.to_datetime(df_sales['Date'], format='%d/%m/%Y')
df_features['Date'] = pd.to_datetime(df_features['Date'], format='%d/%m/%Y')
# Information on the SettingWithCopywarning that might occur: 
# https://www.dataquest.io/blog/settingwithcopywarning/

## Data Exploration and Manipulation

---

Before we build a model, it is always a good idea to take a look at the data itself. Some questions that you might ask when exploring your data are:

- What information is recorded (what do different columns contain)?
- How frequently (since we are working with time series data) is the data recorded?
- How many missing values are there in each column? 
- What are the distributions of the different columns? 
- Are any columns highly correlated to one another?

After exploration, if there are lots of missing values or if the distribution of a column is highly left/right skewed, which could potentially affect the accruacy of your models, you would want to think about replacing or transforming them before you build, compare, and select your models.

### Exploration

First, let's take a look at the `sales` dataframe:

In [3]:
display(df_sales.head(10))
display(df_sales.info())
display(df_sales.describe(include='all').round(2))

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
5,1,1,2010-03-12,21043.39,False
6,1,1,2010-03-19,22136.64,False
7,1,1,2010-03-26,26229.21,False
8,1,1,2010-04-02,57258.43,False
9,1,1,2010-04-09,42960.91,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 13.3 MB


None

,Store,Dept,Date,Weekly_Sales,IsHoliday
count,421570.00,421570.00,421570,421570.00,421570
unique,NaN,NaN,143,NaN,2
top,NaN,NaN,2011-12-23 00:00:00,NaN,False
freq,NaN,NaN,3027,NaN,391909
first,NaN,NaN,2010-02-05 00:00:00,NaN,NaN
last,NaN,NaN,2012-10-26 00:00:00,NaN,NaN
mean,22.20,44.26,NaN,15981.26,NaN
std,12.79,30.49,NaN,22711.18,NaN
min,1.00,1.00,NaN,-4988.94,NaN
25%,11.00,18.00,NaN,2079.65,NaN


We can see that:

1. There are 143 unique dates (421,570 records in total) from 2010-02-05 to 2012-10-26 in the `sales` dataframe. We should keep in mind at this point that, on each day, sales across multiple stores with several departments are recorded, even though we do not know if all stores or departments are open on each of the 143 days. 
2. There is no missing values in any column of this dataframe, which is good.
3. However, we should notice that the `weekly sales` column is extremely right skewed, with a mean of \\$15,981, but a 50th percentile (mostly the same as the median) of only \\$7,612. The range of the column is also fairly large, from -\\$4,989 to \\$693,099.
4. 391,909 records out of 421,570 are not holidays (or with some caculations, about 7% of the records *are* holidays), which could potentially explain why the `weekly sales` records are extremely right skewed (*maybe* most of the sales happen on holidays).

Now let's move onto the `features` dataframe:

In [4]:
display(df_features.head(10))
display(df_features.info())
display(df_features.describe(include='all').round(2))

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
5,1,2010-03-12,57.79,2.667,NaN,NaN,NaN,NaN,NaN,211.380643,8.106,False
6,1,2010-03-19,54.58,2.720,NaN,NaN,NaN,NaN,NaN,211.215635,8.106,False
7,1,2010-03-26,51.45,2.732,NaN,NaN,NaN,NaN,NaN,211.018042,8.106,False
8,1,2010-04-02,62.27,2.719,NaN,NaN,NaN,NaN,NaN,210.820450,7.808,False
9,1,2010-04-09,65.86,2.770,NaN,NaN,NaN,NaN,NaN,210.622857,7.808,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         8190 non-null   int64         
 1   Date          8190 non-null   datetime64[ns]
 2   Temperature   8190 non-null   float64       
 3   Fuel_Price    8190 non-null   float64       
 4   MarkDown1     4032 non-null   float64       
 5   MarkDown2     2921 non-null   float64       
 6   MarkDown3     3613 non-null   float64       
 7   MarkDown4     3464 non-null   float64       
 8   MarkDown5     4050 non-null   float64       
 9   CPI           7605 non-null   float64       
 10  Unemployment  7605 non-null   float64       
 11  IsHoliday     8190 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(9), int64(1)
memory usage: 712.0 KB


None

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
count,8190.00,8190,8190.00,8190.00,4032.00,2921.00,3613.00,3464.00,4050.00,7605.00,7605.00,8190
unique,NaN,182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,2011-07-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
freq,NaN,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7605
first,NaN,2010-02-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2013-07-26 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,23.00,NaN,59.36,3.41,7032.37,3384.18,1760.10,3292.94,4132.22,172.46,7.83,NaN
std,12.99,NaN,18.68,0.43,9262.75,8793.58,11276.46,6792.33,13086.69,39.74,1.88,NaN
min,1.00,NaN,-7.29,2.47,-2781.45,-265.76,-179.26,0.22,-185.17,126.06,3.68,NaN
25%,12.00,NaN,45.90,3.04,1577.53,68.88,6.60,304.69,1440.83,132.36,6.63,NaN


Right away, we can see that:

1. Compared to the `sales` dataframe, there are less records (8190) in the `features` dataframe. It makes sense because this dataframe is only detailed to store level and not department; for example, different departments in the same store *probably* experience the same weather on the same day. 
2. However, with only store level data, we might find it harder to model the effect of markdowns across different departments within the same store. 
3. The are 182 unique dates from 2010-02-05 to 2013-07-26, which is a wider range of record than the `sales` dataframe.
4. From the first 10 rows, we can see that the dates do match up; however, it wouldn't hurt to check the proportion of matching records (by date and store) especially if we are thinking about merging together the two dataframes
5. The 5 mark down columns contain 'anonymous data related to promotional mark downs' as per Kaggle, and we might want to further inspect these columns (e.g. whether two mark downs occured at any point for the same store). 
6. Again, approximately 7% of the 8190 `features` records are holidays, which is consistent with the `sales` dataframe

Last but not least, let's take a look at the `stores` dataframe:

In [5]:
display(df_stores.head())
display(df_stores.info())
display(df_stores.describe(include='all').round(2))

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


None

,Store,Type,Size
count,45.00,45,45.00
unique,NaN,3,NaN
top,NaN,A,NaN
freq,NaN,22,NaN
mean,23.00,NaN,130287.60
std,13.13,NaN,63825.27
min,1.00,NaN,34875.00
25%,12.00,NaN,70713.00
50%,23.00,NaN,126512.00
75%,34.00,NaN,202307.00


The `stores` dataframe appear to be smaller and simper. We can see that:

1. Almost half of the stores are type A stores; we might want to explore later whether there is any correlation between `store type` and other variables in the three dataframes
2. `Sizes` of the stores are not extremely skewed, with its mean close to its 50th percentile. However, we might want to check if they are light or heavy tailed.

Now that we have taken a look at the aggregated data, we want to look at the sales of each store and department on an individual level. However, it is not the most efficient to print or plot the subsets one by one. Luckily, the `interact()` function from the [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) package allow us to create an interactive "dashboard" in Jupyter notebooks:

In [26]:
def sales_viz(store_num, dept_num):
    sales_temp = df_sales[(df_sales.Store==store_num) & (df_sales.Dept==dept_num)][['Date', 'Weekly_Sales']]
    ts_sales_temp = sales_temp.set_index('Date')
    return plt.show(ts_sales_temp.plot(figsize=(10,5))), display(ts_sales_temp.describe().round(2))

interact(sales_viz, store_num={n:n for n in range(1,46)}, dept_num={n:n for n in range(1,100)})

interactive(children=(Dropdown(description='store_num', options={1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: …

<function __main__.sales_viz(store_num, dept_num)>

### Replacement and Transformation